In [22]:
from collections import namedtuple
# neptune
import os
import numpy as np

from time import sleep
import torch
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.nn import functional as F
from models.t5 import *
from data.data import *

In [23]:
model = T5FinetuneForRACE.load_from_checkpoint("D:/Github/decepticon/project/checkpoints/0_02.ckpt")

In [4]:
class Args:
    def __init__(self):
        self.data_path =  "D:\Github/decepticon/Processed_New"
        self.batch_size =  16
        self.num_workers =  0
        self.pretrained_model = "t5-small"
        self.tokenizer_len = 32102 ## manual
        self.padding_id = 0
    def to_dict(self):
        out = dict()
        out["data_path"] = self.data_path
        out["batch_size"] = self.batch_size
        out["num_workers"] = self.num_workers
        out["pretrained_model"] = self.pretrained_model
        out["tokenizer_len"] = self.tokenizer_len
        out["padding_id"] = self.padding_id
        return out

hparams=Args()

In [5]:
data_module = RaceDataModule(hparams)
print(len(data_module.tokenizer))
data_module.prepare_data()
data_module.setup()

32102
SETUP: Training Dataset
SETUP: Validation Dataset
SETUP: Test Dataset


In [41]:
def generate_question(model, content):
    model.eval()        
    #output = model.model.generate(input_ids=content["input_ids"])
    output = model.model.generate(input_ids = content["input_ids"], 
                                  num_beams = 5,
                                  max_lengths = 50,
                                  early_stopping = True)
    decode = data_module.tokenizer.decode(output[0], skip_special_tokens= True, clean_up_tokenization_spaces = True)
    content = [data_module.tokenizer.decode(i) for i in content["input_ids"]]
    return decode, content

In [43]:
i = 0
for x, y in data_module.val_dataloader():
    output = generate_question(model, x)
    print("-----------------------------------------")
    print("GROUND TRUTH: %s" %data_module.tokenizer.decode(y["input_ids"][0], skip_special_tokens= True, clean_up_tokenization_spaces = True))
    print("GENERATED: %s"  %"".join(output[0]))
    print("ANSWER: %s" % "".join(output[1]).split("<context>")[0].replace("<answer>", "").lstrip())
    i += 1
    if i > 10: break

-----------------------------------------
GROUND TRUTH: The Sherman Antitrust Act _.
GENERATED: The Sherman Antitrust Act _.
ANSWER: sought to eliminate monopolies in favor of competition in the market-place
-----------------------------------------
GROUND TRUTH: Why are ECAs important now?
GENERATED: What does the passage mainly talk about?
ANSWER: They prepare the students for their future.
-----------------------------------------
GROUND TRUTH: Which of the following should be avoided when taking selfies?
GENERATED: Which of the following can make the photo appealing?
ANSWER: A busy background.
-----------------------------------------
GROUND TRUTH: What can be the subject of this passage?
GENERATED: Which of the following is TRUE according to the passage?
ANSWER: People across the world get married in different ways and for different reasons.
-----------------------------------------
GROUND TRUTH: Why didn't the fish eat our night crawlers?
GENERATED: Which of the following is TRUE